In [10]:
from pulp import *

#initialize the problem
transportation = LpProblem("Transportation_Problem", LpMinimize)

#data: lists of plants and customers
plants = ["Toronto", "Detroit"] 
customers = ["NYC", "Philly", "StLouis"] 
warehouses = ["Chicago","Buffalo"]

#data: dictionaries of supply, demand, and transportation cost. 

supply = {"Toronto":800, "Detroit":700} 
demand = {"NYC":450, "Philly":350, "StLouis":300} 

tcostpw= {"Toronto":{"Chicago":4, "Buffalo":7},
          "Detroit":{"Chicago":5, "Buffalo":7}} 
tcostwc= {"Chicago":{"NYC":6, "Philly":4, "StLouis":5},
          "Buffalo":{"NYC":2,"Philly":3,"StLouis":4}}

#variables
x = LpVariable.dicts("quantity_shipped", (plants, warehouses), lowBound=0) 
y = LpVariable.dicts("quantity_shipped", (warehouses,customers), lowBound=0)

#obj. function
transportation += lpSum(tcostpw[i][j]*x[i][j] for i in plants for j in warehouses) + lpSum(tcostwc[i][j]*y[i][j] for i in warehouses for j in customers) 

#supply constraints
for i in plants:
	transportation += lpSum(x[i][j] for j in warehouses)<=supply[i]
#demand constraints
for j in customers:
	transportation +=lpSum(y[i][j] for i in warehouses)==demand[j]

for w in warehouses:
    transportation += lpSum(x[i][w] for i in plants) == lpSum(y[w][i] for i in customers)

transportation.solve(PULP_CBC_CMD(msg=0))


print("Status= ", LpStatus[transportation.status])

print("Total Cost= ", value(transportation.objective))

for v in transportation.variables():
    if v.value()>0:
        print(v.name, " = ", v.value())

print("\n\n LP model:\n ")
print(transportation)







Status=  Optimal
Total Cost=  9550.0
quantity_shipped_Buffalo_NYC  =  450.0
quantity_shipped_Chicago_Philly  =  350.0
quantity_shipped_Chicago_StLouis  =  300.0
quantity_shipped_Detroit_Buffalo  =  450.0
quantity_shipped_Toronto_Chicago  =  650.0


 LP model:
 
Transportation_Problem:
MINIMIZE
2*quantity_shipped_Buffalo_NYC + 3*quantity_shipped_Buffalo_Philly + 4*quantity_shipped_Buffalo_StLouis + 6*quantity_shipped_Chicago_NYC + 4*quantity_shipped_Chicago_Philly + 5*quantity_shipped_Chicago_StLouis + 7*quantity_shipped_Detroit_Buffalo + 5*quantity_shipped_Detroit_Chicago + 7*quantity_shipped_Toronto_Buffalo + 4*quantity_shipped_Toronto_Chicago + 0.0
SUBJECT TO
_C1: quantity_shipped_Toronto_Buffalo + quantity_shipped_Toronto_Chicago
 <= 800

_C2: quantity_shipped_Detroit_Buffalo + quantity_shipped_Detroit_Chicago
 <= 700

_C3: quantity_shipped_Buffalo_NYC + quantity_shipped_Chicago_NYC = 450

_C4: quantity_shipped_Buffalo_Philly + quantity_shipped_Chicago_Philly = 350

_C5: quantity_sh